# Instalación de dependencias

Librerías de Python necesarias para correr el modelo. Solo es necesario instalarlas la primera vez que se usa el sistema.

In [ ]:
%pip install ultralytics
%pip install imageio imageio-ffmpeg
%pip install numpy
%pip install pillow
%pip install mss
%pip install pygame
%pip install opencv-python

# Inicialización de modelo

Importa las librerías necesarias y prende el modelo de detección. Es necesario correrlo al inicio de cada sesión de uso.

In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
import cv2
from concurrent.futures import ThreadPoolExecutor
import pygame
import mss

model = YOLO('MOY_YOLO.pt')


# Detección de imagen

Toma como entrada una imagen .png, .jpg, o .webp y aplica el motor de inferencia sobre esta. Se muestran los cuadros de delimitación encontrados en la imagen.

In [ ]:
imagen = 'sample.jpg' # Reemplazar con el nombre de la imagen a predecir, incluyendo la extensión del archivo
infer = model
results = infer.predict(f"{imagen}")
for r in results:
  boxes = r.boxes
  annots = []
  for box in boxes:
    annot = []
    annot.append(box.xyxy[0][0].item())
    annot.append(box.xyxy[0][1].item())
    annot.append(box.xyxy[0][2].item())
    annot.append(box.xyxy[0][3].item())
    annots.append(annot)
img = Image.open(f"{imagen}")
draw = ImageDraw.Draw(img)
for annot in annots:
  draw.rectangle([annot[0],annot[1],annot[2],annot[3]], outline="red", width=5)
display(img)
img.save(f"./predicted.png") #esta linea puede ser comentada si solo se desea ver la prediccion sin guardarla

# Detección de Video

La siguiente celda inicializa la detección de video mediante la cámara del dispositivo donde se esté corriendo el sistema. Presionar Q para finalizar.

In [ ]:
video_path = 0 
reader = cv2.VideoCapture(video_path)
fps = reader.get(cv2.CAP_PROP_FPS)
frame_width = int(reader.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

pygame.mixer.init()
pygame.mixer.music.load('alarm.mp3')

def process_frame(frame):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    results = model(img)
    annots = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            annot = [
                box.xyxy[0][0].item(),
                box.xyxy[0][1].item(),
                box.xyxy[0][2].item(),
                box.xyxy[0][3].item()
            ]
            annots.append(annot)
    return annots

def draw_annotations(frame, annotations):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    for annot in annotations:
        draw.rectangle([annot[0], annot[1], annot[2], annot[3]], outline="red", width=5)
    annotated_frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    return annotated_frame

def read_frame():
    ret, frame = reader.read()
    return frame if ret else None

skip_frames = max(1, int(fps / 1))
annotations = []
playing = False

with ThreadPoolExecutor(max_workers=100) as executor:
    frame_count = 0
    while True:
        frame = read_frame()
        if frame is None:
            break
        
        if frame_count % skip_frames == 0:
            future = executor.submit(process_frame, frame)
            annotations = future.result()
            # Play audio if there are detections
            if annotations and not playing:
                pygame.mixer.music.play(-1)  # Loop the alarm
                playing = True
            elif not annotations and playing:
                pygame.mixer.music.stop()
                playing = False
        
        annotated_frame = draw_annotations(frame, annotations)
        cv2.imshow('Video Stream', annotated_frame)
        # writer.write(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1

reader.release()
cv2.destroyAllWindows()
pygame.mixer.music.stop()

La siguiente celda inicializa la detección de video sobre lo que se muestra en la pantalla. Se detectará la mitad izquierda de la pantalla, por lo cual se recomienta ubicar la ventana de output al lado derecho. Presionar Q para finalizar.

In [ ]:
pygame.mixer.init()
pygame.mixer.music.load('alarm.mp3')

sct = mss.mss()

monitor = sct.monitors[1]

def capture_screen():
    sct_img = sct.grab(monitor)
    img = np.array(sct_img)
    frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    frame[:, frame.shape[1] // 2:] = 0
    return frame

fps = 10
skip_frames = max(1, int(fps / 1))
annotations = []
playing = False

def process_frame(frame):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    results = model(img)
    annots = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            annot = [
                box.xyxy[0][0].item(),
                box.xyxy[0][1].item(),
                box.xyxy[0][2].item(),
                box.xyxy[0][3].item()
            ]
            annots.append(annot)
    return annots

def draw_annotations(frame, annotations):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    for annot in annotations:
        draw.rectangle([annot[0], annot[1], annot[2], annot[3]], outline="red", width=5)
    annotated_frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    return annotated_frame

with ThreadPoolExecutor(max_workers=100) as executor:
    frame_count = 0
    while True:
        frame = capture_screen()
        
        if frame_count % skip_frames == 0:
            future = executor.submit(process_frame, frame)
            annotations = future.result()
            if annotations and not playing:
                pygame.mixer.music.play(-1)
                playing = True
            elif not annotations and playing:
                pygame.mixer.music.stop()
                playing = False
        
        annotated_frame = draw_annotations(frame, annotations)
        cv2.imshow('Screen Stream', annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1

cv2.destroyAllWindows()
pygame.mixer.music.stop()
